# EP Tender : simulation par chaînes de Markov

## Importation des modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt

On importe aussi les fonctions définies dans le fichier Specifications_Modele

In [3]:
# !pip install ipynb
from ipynb.fs.full.Specifications_modele import *

## Graphiques

### Graphiques gloutons

In [5]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, nCharge)
ajout = rangementGloutonSimplifie
retrait = lambda x : donGloutonSimplifie(x, nCharge)
charge = lambda x : chargeSimplifie(x, nCharge)
theta = 5/6

M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1

v, _ = expectedHittingTime(M, i, j)

print(v)

7.601282656784576


In [ ]:
thetaArray, expectedHittingTimes = plotExpectedHittingTime(nTenders, nFiles, nCharge, rangementGloutonSimplifie, donGloutonSimplifie, chargeSimplifie, res = 0.01, etatDepart = etatDepart)

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.05, 0.05, 0.95, 0.95])

ax.plot(thetaArray, expectedHittingTimes, color = blue)
ax.set_ylim([0, 50])

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.05, 0.05, 0.95, 0.95])
Y = []

for nFile in [2, 3, 4] :
    thetaArray, expectedHittingTimes = plotExpectedHittingTime(6, nFile, 2, rangementGloutonSimplifie, donGloutonSimplifie, chargeSimplifie, res = 0.1)
    Y.append(expectedHittingTimes)
    print(nFile)
    
ax.plot(thetaArray, np.array(Y).transpose())
ax.set_ylim([0, 50])
ax.legend(['2 files', '3 files', '4 files'])
ax.set_title('6 tenders, 2 niveaux de charge')
plt.show()

### Graphiques pour indice linéaire

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

lbdArray = np.arange(1, 10 + 0.25, 0.25)
tabTErr = []

for lbd in lbdArray :
    indice = lambda x : indiceLineaire(x, lbd)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)
    
    etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
    etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
    etatDepart += [[0, 0]]
    etatDepart = np.array(etatDepart)
    
    etatArrivee = False
    
    i = 0
    j = 0

    while not np.array(listeEtat[i] == etatDepart).all() :
        i += 1

    while not np.array(listeEtat[j] == etatArrivee).all() :
        j += 1
    
    tabTErr.append(expectedHittingTime(M, i, j)[0])

print(tabTErr)

In [ ]:
Y = [5.47791836505727,
 6.030111546934522,
 6.355876349465076,
 6.742220604119133,
 6.961440015488954,
 7.251570899599649,
 7.371388850799515,
 7.541218390033908,
 7.576264818604656,
 7.656724262102546,
 7.663366109338309,
 7.690640704517898,
 7.693288240814784,
 7.6814874478769415,
 7.6821618376615515,
 7.668450304043451,
 7.668450304043451,
 7.647648488095228,
 7.647648488095228]

X = [ 1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,  6. ,
        6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10.]

savedCSV = []
for i in range(len(X)) :
    savedCSV += [[X[i], Y[i]]]
    
np.savetxt('indiceLin.csv',
            savedCSV,
            delimiter = ',',
            fmt = ['%f', '%f'],
            header = 'Lambda,TErr')

In [ ]:
lbdArray[::2]

In [ ]:
tabTErr[::2]

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(lbdArray[::2], tabTErr[::2])

ax.set_ylabel("Temps moyen jusqu'à l'arrivée du premier client qu'on\nne peut pas immédiatement servir (en heures)")
ax.set_xlabel("Valeur du paramètre $\lambda$")
ax.set_xlim([0, 10])
ax.set_ylim([6, 8])
ax.set_yticks(np.arange(6, 8.5, 0.5))

ax.axhline(v, color = 'black', linestyle = 'dashed', linewidth = 1)
ax.text(s = "Valeur pour l'algorithme glouton :\n" + str(round(v,1)) + ' heures', x = 0.2, y = v - 0.15)

ax.axhline(tabTErr[-13], color = 'red', linestyle = 'dashed', linewidth = 1, xmax = lbdArray[-13] / 10)
ax.text(s = "Valeur à l'optimum :\n" + str(round(tabTErr[-13],1)) + ' heures', color = 'red', x = 0.2, y = tabTErr[-13] + 0.05)

ax.axvline(lbdArray[-13], color = 'red', linestyle = 'dashed', linewidth = 1, ymax = (tabTErr[-13] - 6) / 2)
ax.text(s = "$\lambda$ optimal : " + str(round(lbdArray[-13],1)), color = 'red', x = lbdArray[-13] + 0.2, y = 6.2)

fig.show()

### Graphiques pour indice logarithmique

In [ ]:
tabTErr

In [ ]:
indice = lambda x : indiceLogarithmique(x, 1)
ajout = lambda x : rangementIndiceSimplifie(x, indice)
M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1

print(expectedHittingTime(M, i, j)[0])

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

lbdArray = np.arange(1.5, 8.5, 0.5)
tabTErr = []

for lbd in lbdArray :
    indice = lambda x : indiceLogarithmique(x, lbd)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)
    
    etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
    etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
    etatDepart += [[0, 0]]
    etatDepart = np.array(etatDepart)
    
    etatArrivee = False
    
    i = 0
    j = 0

    while not np.array(listeEtat[i] == etatDepart).all() :
        i += 1

    while not np.array(listeEtat[j] == etatArrivee).all() :
        j += 1
    
    tabTErr.append(expectedHittingTime(M, i, j)[0])

print(tabTErr)

In [ ]:
np.savetxt('indiceLog.csv',
            [(lbdArray[i], tabTErr[i]) for i in range(len(tabTErr))],
            delimiter = ',',
            fmt = ['%f', '%f'])

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(lbdArray, tabTErr)

ax.set_ylabel("Temps moyen jusqu'à l'arrivée du premier client qu'on\nne peut pas immédiatement servir (en heures)")
ax.set_xlabel("Valeur du paramètre $\lambda$")
ax.set_xlim([0, 10])
ax.set_ylim([6, 8])
ax.set_yticks(np.arange(6, 8.5, 0.5))

ax.axhline(v, color = 'black', linestyle = 'dashed', linewidth = 1)
ax.text(s = "Valeur pour l'algorithme glouton :\n" + str(round(v,2)) + ' heures', x = 0.2, y = v - 0.15)

ax.axhline(tabTErr[-8], color = 'red', linestyle = 'dashed', linewidth = 1, xmax = lbdArray[-8] / 10)
ax.text(s = "Valeur à l'optimum :\n" + str(round(tabTErr[-8],2)) + ' heures', color = 'red', x = 0.2, y = tabTErr[-8] + 0.05)

ax.axvline(lbdArray[-8], color = 'red', linestyle = 'dashed', linewidth = 1, ymax = (tabTErr[-8] - 6) / 2)
ax.text(s = "$\lambda$ optimal : " + str(round(lbdArray[-8],2)), color = 'red', x = lbdArray[-8] + 0.2, y = 6.2)

fig.show()

### Graphiques pour indice multiplicatif

In [ ]:
etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

lbdArray = np.arange(0, 3.1, 0.1)
tabTErr = []

for lbd in lbdArray :
    indice = lambda x : indiceMultiplicatif(x, lbd)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    
    tabTErr.append(expectedHittingTime(M, i, j)[0])

print(tabTErr)

In [ ]:
np.savetxt('indiceMult.csv',
            [(lbdArray[i], tabTErr[i]) for i in range(len(tabTErr))],
            delimiter = ',',
            fmt = ['%f', '%f'])

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(lbdArray, tabTErr)

ax.set_ylabel("Temps moyen jusqu'à l'arrivée du premier client qu'on\nne peut pas immédiatement servir (en heures)")
ax.set_xlabel("Valeur du paramètre $\lambda$")
ax.set_xlim([0, 3])
ax.set_ylim([6, 8])
ax.set_yticks(np.arange(6, 8.5, 0.5))

fig.show()

### Graphiques pour indice multiplicatif ajusté

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

lbdArray = np.arange(0, 2.1, 0.2)
tabTErr = []

for lbd in lbdArray :
    indice = lambda x : indiceMultiplicatifCorr(x, lbd, d)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)
    
    etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
    etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
    etatDepart += [[0, 0]]
    etatDepart = np.array(etatDepart)
    
    etatArrivee = False
    
    i = 0
    j = 0

    while not np.array(listeEtat[i] == etatDepart).all() :
        i += 1

    while not np.array(listeEtat[j] == etatArrivee).all() :
        j += 1
    
    tabTErr.append(expectedHittingTime(M, i, j)[0])

print(tabTErr)

In [ ]:
np.savetxt('indiceMultCorr.csv',
            [(lbdArray[i], tabTErr[i]) for i in range(len(tabTErr))],
            delimiter = ',',
            fmt = ['%f', '%f'])

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(lbdArray[::1], tabTErr[::1])

ax.set_ylabel("Temps moyen jusqu'à l'arrivée du premier client qu'on\nne peut pas immédiatement servir (en heures)")
ax.set_xlabel("Valeur du paramètre $\lambda$")
ax.set_xlim([0, 2])
ax.set_ylim([6, 8])
ax.set_yticks(np.arange(6, 8.5, 0.5))

fig.show()

### Graphiques pour indice puissance-logarithme

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1

In [ ]:
# On fractionne le calcul en deux et on change lambda à la main (valeurs : 0.2, 1/3, 0.5, 1, 2, 3)
# pour diviser le calcul en plusieurs calculs plus courts

lbd = 1/5

tempTab = []
muArray = np.arange(0, 1, 0.1).tolist()

for mu in muArray :
    indice = lambda x : indicePuisLog(x, lbd, mu)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr.append(tempTab)

In [ ]:
lbd = 1/5

tempTab = []
muArray = np.arange(1, 11, 1).tolist()

for mu in muArray :
    indice = lambda x : indicePuisLog(x, lbd, mu)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr[-1] = tabTErr[-1] + tempTab

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist(), np.array(tabTErr).transpose())

fig.show()

In [ ]:
muArray = np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist()

np.savetxt('indicePuisLog.csv',
            [tuple([muArray[i]]) +tuple(np.array(tabTErr).transpose()[i]) for i in range(len(muArray))],
            delimiter = ',',
            fmt = ['%f'] * (len(tabTErr) + 1))

### Graphiques pour indice puissance-log inversés

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1
    
tabTErr = []

In [ ]:
# On fractionne le calcul en deux et on change lambda à la main (valeurs : 0.2, 1/3, 0.5, 1, 2, 3)
# pour diviser le calcul en plusieurs calculs plus courts

lbd = 3

tempTab = []
muArray = np.arange(0, 1, 0.1).tolist()

for mu in muArray :
    indice = lambda x : indicePuisLogCorr(x, lbd, mu, d)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr.append(tempTab)

In [ ]:
lbd = 3

tempTab = []
muArray = np.arange(1, 11, 1).tolist()

for mu in muArray :
    indice = lambda x : indicePuisLogCorr(x, lbd, mu, d)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr[-1] = tabTErr[-1] + tempTab

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist(), np.array(tabTErr).transpose())

fig.show()

In [ ]:
muArray = np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist()

np.savetxt('indicePuisLogInv.csv',
            [tuple([muArray[i]]) +tuple(np.array(tabTErr).transpose()[i]) for i in range(len(muArray))],
            delimiter = ',',
            fmt = ['%f'] * (len(tabTErr) + 1))

### Indices puissance

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1
    
tabTErr = []

In [ ]:
# On fractionne le calcul en deux et on change lambda à la main (valeurs : 0.2, 1/3, 0.5, 1, 2, 3)
# pour diviser le calcul en plusieurs calculs plus courts

lbd = 3

tempTab = []
muArray = np.arange(0, 1, 0.1).tolist()

for mu in muArray :
    indice = lambda x : indicePuis(x, lbd, mu)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr.append(tempTab)

In [ ]:
lbd = 3

tempTab = []
muArray = np.arange(1, 11, 1).tolist()

for mu in muArray :
    indice = lambda x : indicePuis(x, lbd, mu)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr[-1] = tabTErr[-1] + tempTab

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist(), np.array(tabTErr).transpose())

ax.set_xlim([0, 1])

fig.show()

In [ ]:
muArray = np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist()

np.savetxt('indicePuis.csv',
            [tuple([muArray[i]]) +tuple(np.array(tabTErr).transpose()[i]) for i in range(len(muArray))],
            delimiter = ',',
            fmt = ['%f'] * (len(tabTErr) + 1))

### Indices puissance inversés

In [ ]:
nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)
theta = 5/6

etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1
    
tabTErr = []

In [ ]:
# On fractionne le calcul en deux et on change lambda à la main (valeurs : 0.2, 1/3, 0.5, 1, 2, 3)
# pour diviser le calcul en plusieurs calculs plus courts

lbd = 3

tempTab = []
muArray = np.arange(0, 1, 0.1).tolist()

for mu in muArray :
    indice = lambda x : indicePuisCorr(x, lbd, mu, d)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr.append(tempTab)

In [ ]:
lbd = 3

tempTab = []
muArray = np.arange(1, 11, 1).tolist()

for mu in muArray :
    indice = lambda x : indicePuisCorr(x, lbd, mu, d)
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)

    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr[-1] = tabTErr[-1] + tempTab

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist(), np.array(tabTErr).transpose())

ax.set_xlim([0, 1])

fig.show()

In [ ]:
muArray = np.arange(0, 1, 0.1).tolist() + np.arange(1, 11, 1).tolist()

np.savetxt('indicePuisCorr.csv',
            [tuple([muArray[i]]) +tuple(np.array(tabTErr).transpose()[i]) for i in range(len(muArray))],
            delimiter = ',',
            fmt = ['%f'] * (len(tabTErr) + 1))

### Sensibilité à une méspécification des paramètres

In [ ]:
tabIndices = []

nTenders = 20
nFiles = 3
nCharge = 2
listeEtat = listAllStations(nTenders, nFiles, d)
retrait = lambda x : donGloutonSimplifie(x, d)
charge = lambda x : chargeSimplifie(x, d)

rhoArray = [1/2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
thetaArray = [rho / (rho + 1) for rho in rhoArray]

indice = lambda x : indiceLineaire(x, 30)
tabIndices.append(indice)

indice = lambda x : indiceLineaire(x, 7)
tabIndices.append(indice)

indice = lambda x : indiceLogarithmique(x, 3.5)
tabIndices.append(indice)

indice = lambda x : indiceMultiplicatif(x, 0.6)
tabIndices.append(indice)

indice = lambda x : indicePuis(x, 0.5, 0.1)
tabIndices.append(indice)

indice = lambda x : indicePuisCorr(x, 3, 0.2, d)
tabIndices.append(indice)

indice = lambda x : indicePuisLog(x, 0.2, 0.1)
tabIndices.append(indice)

indice = lambda x : indicePuisLogCorr(x, 3, 0.9, d)
tabIndices.append(indice)

tabTErr = []

In [ ]:
etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
etatDepart += [[0, 0]]
etatDepart = np.array(etatDepart)

etatArrivee = False

i = 0
j = 0

while not np.array(listeEtat[i] == etatDepart).all() :
    i += 1

while not np.array(listeEtat[j] == etatArrivee).all() :
    j += 1

In [ ]:
indexTheta = 13
theta = thetaArray[indexTheta]

tempTab = []
for indice in tabIndices :
    ajout = lambda x : rangementIndiceSimplifie(x, indice)
    M = createTransitionMatrix(listeEtat, ajout, retrait, charge, theta)
    tempTab.append(expectedHittingTime(M, i, j)[0])

tabTErr.append(tempTab)

In [ ]:
x = tabTErr[8]

In [ ]:
[np.sign(x[1:] - x[0]) for x in tabTErr]

In [ ]:
fig = plt.figure()
ax = fig.add_axes(rect = [0.1, 0.1, 0.9, 0.9])

ax.plot(rhoArray[2:11], [np.log(abs(x[1:] - x[0]) / x[0]) for x in [y[:6] + y[7:] for y in tabTErr[2:11]]])

fig.show()

In [ ]:
np.savetxt('algRobusteNice.csv',
           [tuple([rhoArray[2:11][i]]) + tuple([100 * abs(x[1:] - x[0]) / x[0] for x in [y[:6] + y[7:] for y in tabTErr[2:11]]][i]) for i in range(9)],
            delimiter = ',',
            fmt = ['%f'] * (len(tabTErr[0]) - 1))

In [ ]:
np.savetxt('algRobuste.csv',
            [tuple([rhoArray[i]]) +tuple(np.array(tabTErr)[i]) for i in range(len(rhoArray))],
            delimiter = ',',
            fmt = ['%f'] * (len(tabTErr[0]) + 1))